# 1

## Many-to-One Sequence Problems with a Single Feature

In [ ]:
"""
In many-to-one sequence problems, each input sample has more than one time-step, however the output consists of a single element. 
Each time-step in the input can have one or more features.

Let's first create the dataset. Our dataset will consist of 15 samples. Each sample will have 3 time-steps where each time-step will 
consist of a single feature i.e. a number. The output for each sample will be the sum of the numbers in each of the three time-steps. 
For instance, if our sample contains a sequence 4,5,6 the output will be 4 + 5 + 6 = 10.

"""

X = np.array([x+1 for x in range(45)])
print(X)

X = X.reshape(15,3,1)
print(X)
"""
[[[ 1]
  [ 2]
  [ 3]]

 [[ 4]
  [ 5]
  [ 6]]
  .
  .
  ]
"""

"""
each element in the output will be equal to the sum of the values in the time-steps in the corresponding input sample.
"""
Y = list()
for x in X:
    Y.append(x.sum())

Y = np.array(Y)
print(Y)

"""
[  6  15 . . . . . ]
"""

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

test_input = array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

## Stacked LSTM

In [ ]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(3, 1)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

## Bidirectional LSTM

In [ ]:
"""
Bidirectional LSTM is a type of LSTM which learns from the input sequence from both forward and backward directions. 
The final sequence interpretation is the concatenation of both forward and backward learning passes.
"""

from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
test_output = model.predict(test_input, verbose=0)
print(test_output)

# 2

## Many-to-one Sequence Problems with Multiple Features

In [ ]:
"""
In a many-to-one sequence problem we have an input where each time-steps consists of multiple features. The output can be a single value 
or multiple values, one per feature in the input time step.

dataset will contain 15 samples. Each sample will consist of 3 time-steps. Each time-steps will have two features.

Let's create two lists. One will contain multiples of 3 until 135 i.e. 45 elements in total. The second list will contain multiples of 5,
from 1 to 225. The second list will also contain 45 elements in total.
"""

X1 = np.array([x+3 for x in range(0, 135, 3)])
print(X1)

X2 = np.array([x+5 for x in range(0, 225, 5)])
print(X2)

"""
The aggregated dataset can be created by joining the two lists
"""
X = np.column_stack((X1, X2))
print(X)

"""
We need to reshape our data into three dimensions so that it can be used by LSTM. We have 45 rows in total and two columns in our dataset. 
We will reshape our dataset into 15 samples, 3 time-steps
"""

"""
The output will also have 15 values corresponding to 15 input samples. Each value in the output will be the sum of the two feature values in 
the third time-step of each input sample. For instance, the third time-step of the first sample has features 9 and 15, hence the output will 
be 24. Similarly, the two feature values in the third time-step of the 2nd sample are 18 and 30; the corresponding output will be 48, and so on.
"""

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

test_input = array([[8, 51],
                    [11,56],
                    [14,61]])

test_input = test_input.reshape((1, 3, 2))
test_output = model.predict(test_input, verbose=0)
print(test_output)



## Solution via Stacked LSTM

In [ ]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(3, 2)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

test_output = model.predict(test_input, verbose=0)
print(test_output)

## Solution via Bidirectional LSTM

In [ ]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(3, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
test_output = model.predict(test_input, verbose=0)
print(test_output)

# 3

In [ ]:
"""
Till now we have predicted single values based on multiple features values from different time-steps. There is another case of many-to-one 
sequences where you want to predict one value for each feature in the time-step. For instance, the dataset we used in this section has three 
time-steps and each time-step has two features. We may want to predict individual values for each feature series.
"""

"""
[[[  3   5]
  [  6  10]
  [  9  15]]
In the output, we want one time-step with two features as shown below:

[12, 20]

the first value in the output is a continuation of the first series and the second value is the continuation of the second series. 
We can solve such problems by simply changing the number of neurons in the output dense layer to the number of features values that 
we want in the output. However, first we need to update our output vector Y. The input vector will remain the same
"""

Y = list()
for x in X:
    new_item = list()
    new_item.append(x[2][0]+3)
    new_item.append(x[2][1]+5)
    Y.append(new_item)

Y = np.array(Y)
print(Y)

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 2)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

test_input = array([[20,34],
                    [23,39],
                    [26,44]])

test_input = test_input.reshape((1, 3, 2))
test_output = model.predict(test_input, verbose=0)
print(test_output)

## Stacked LSTM

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(3, 2)))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=500, validation_split=0.2, verbose=1)

test_output = model.predict(test_input, verbose=0)
print(test_output)

## Bidirectional LSTM

In [ ]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(3, 2)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
test_output = model.predict(test_input, verbose=0)
print(test_output)